<div align="center">
  <img src="../assets/images/hackathon.png" alt="Holistic AI Hackathon Logo" width="600"/>
</div>

**Event**: [hackathon.holisticai.com](https://hackathon.holisticai.com)

---


# Tutorial 3: Structured Output with Pydantic

**Learn to get JSON responses from your agents**

## What You'll Learn

1. **Define schemas** with Pydantic
2. **Create agents** that return validated JSON
3. **Compare** natural text vs structured output
4. **Best practices** for using structured output

## Why Structured Output?

- **Type safety** - Validated fields
- **Easy parsing** - Direct JSON objects
- **API ready** - Perfect for integrations
- **Reliable** - No parsing errors

---

## Prerequisites

- Basic Python knowledge
- Recommended: Completed `01_basic_agent.ipynb` and `02_custom_tools.ipynb`
- Time: ~15 minutes

**Note:** This tutorial is completely self-contained and uses only official packages!

## Why We Need Structured Output

### The Problem with Natural Text Responses

When you ask an LLM a question, it naturally returns text. But in real applications, you often need:

```python
# What you get from the LLM:
"The temperature is 72 degrees Fahrenheit, humidity is around 65%, and it's partly cloudy."

# What your application needs:
{
  "temperature": 72,
  "temperature_unit": "F",
  "humidity": 65,
  "conditions": "partly cloudy"
}
```

### Real-World Problems Without Structured Output

**Problem 1: Parsing Failures**
```python
# LLM might say any of these:
"The temperature is 72°F"
"It's 72 degrees"
"About seventy-two Fahrenheit"
"72F"

# Your regex will break on most of these!
```

**Problem 2: Missing or Extra Fields**
```python
# You expect: {"name": "...", "age": ..., "email": "..."}
# LLM returns: {"name": "John", "age": 30}  # Missing email!
# Or: {"name": "John", "age": 30, "email": "...", "phone": "..."}  # Extra field!
```

**Problem 3: Type Mismatches**
```python
# You expect: {"score": 85.5}
# LLM returns: {"score": "85.5"}  # String instead of float!
# Or: {"score": "Eighty-five point five"}  # Text instead of number!
```

**Problem 4: Inconsistent Format**
```python
# Request 1: {"date": "2024-01-15"}
# Request 2: {"date": "January 15, 2024"}
# Request 3: {"date": "15/01/2024"}
# All different formats for the same thing!
```

### Why This Matters

In production systems, you need:
- **Reliability** - Code shouldn't break because of formatting changes
- **Type Safety** - Variables must have predictable types
- **Validation** - Data must meet business rules (e.g., age > 0)
- **API Integration** - External systems expect exact schemas
- **Database Storage** - Tables need consistent column types

**Bottom Line:** Natural text is great for humans, but applications need structured data!

import os
from pathlib import Path
from dotenv import load_dotenv

# Load from .env file in parent directory
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables")

# Verify API keys
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded")
else:
    print("  ⚠️  No API keys found")

print("\n📁 Working directory:", Path.cwd())

# Import Holistic AI Bedrock helper
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import get_chat_model
    print("\n✅ Holistic AI Bedrock helper loaded")
except ImportError:
    print("\n⚠️  Could not import from core - will use OpenAI only")

# Import official packages
from pydantic import BaseModel, Field
from typing import List, Optional

print("\n✅ All imports successful!")


### JSON Mode vs Structured Outputs (Comparison)

| Feature | JSON Mode | Structured Outputs |
|---------|-----------|-------------------|
| **Valid JSON Syntax** | - Yes | - Yes |
| **Schema Compliance** | - No guarantee | - 100% guaranteed |
| **Field Names** | - Might be wrong | - Exact match |
| **Field Types** | - Might be wrong | - Type-safe |
| **Validation** | - Manual required | - Automatic |
| **Speed** | Faster | ~10-20% slower |
| **Reliability** |  ~60-80% | - 100% |
| **Use Case** | Simple JSON needed | Strict schema required |

**When to Use JSON Mode:**
- Quick prototyping
- Flexible schema acceptable
- Speed is critical
- Manual validation is okay

**When to Use Structured Outputs:**
- Production APIs
- Database storage
- Type safety required
- Integration with other systems
- You need guarantees

**Now let's see this in action...**

## Step 0: Install Dependencies

Run this cell to install all required packages.

In [59]:
# Install required packages
!pip install -q langgraph langchain-core langchain-openai python-dotenv

print("All dependencies installed!")

All dependencies installed!


## Step 1: Setup Environment

**Recommended:**
Set up Holistic AI Bedrock Proxy API credentials in `.env`:

```bash
HOLISTIC_AI_TEAM_ID=tutorials_api
HOLISTIC_AI_API_TOKEN=your-token-here
```

[API Guide](../assets/api-guide.pdf)

**Alternative (Optional):**
If you prefer OpenAI:

```bash
OPENAI_API_KEY=your-openai-api-key-here
```

**Note:** The tutorial uses Holistic AI Bedrock by default (recommended). OpenAI is optional - to use it, call get_chat_model(model_name, use_openai=True) and set OPENAI_API_KEY.

**Note on Structured Output:** Basic structured output works with Holistic AI Bedrock. For advanced features like `with_structured_output()`, OpenAI may be needed as an alternative option.

In [60]:
import os
import time
import json
from pathlib import Path
from dotenv import load_dotenv

# ============================================
# OPTION 1: Set API keys directly (Quick Start)
# ============================================
# Uncomment and set your keys here:
# Recommended: Holistic AI Bedrock
# os.environ["HOLISTIC_AI_TEAM_ID"] = "tutorials_api"
# os.environ["HOLISTIC_AI_API_TOKEN"] = "your-token-here"
# Alternative: OpenAI (optional)
# os.environ["OPENAI_API_KEY"] = "your-openai-key-here"

# ============================================
# OPTION 2: Load from .env file (Recommended)
# ============================================
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables or hardcoded keys")

# ============================================
# Import Holistic AI Bedrock helper function
# ============================================
# Import from core module (recommended)
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import HolisticAIBedrockChat, get_chat_model
    print("✅ Holistic AI Bedrock helper function loaded")
except ImportError:
    print("⚠️  Could not import from core - will use OpenAI only")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from typing import List

# Verify API keys
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded (Bedrock credentials not set)")
    print("     💡 Tip: Set HOLISTIC_AI_TEAM_ID and HOLISTIC_AI_API_TOKEN to use Bedrock (recommended)")
else:
    print("  ⚠️  No API keys found")
    print("     Set Holistic AI Bedrock credentials (recommended) or OpenAI key")

print("✅ All imports successful!")


📄 Loaded configuration from .env file
⚠️  holistic_ai_bedrock.py not found - will use OpenAI only

🔑 API Key Status:
  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)

✅ All imports successful!


## Step 2: Define a Pydantic Schema

Let's define a structured format for analyzing technologies:

In [61]:
# Define output schema
class TechAnalysis(BaseModel):
    """Structured analysis of a technology."""
    summary: str = Field(description="One-sentence summary")
    key_points: List[str] = Field(description="3-5 key points")
    pros: List[str] = Field(description="2-3 advantages")
    cons: List[str] = Field(description="2-3 disadvantages")
    confidence: float = Field(description="Confidence score 0-100", ge=0, le=100)

print("Schema defined!")
print(f"   Fields: {list(TechAnalysis.model_fields.keys())}")
print(f"   Validation: Built-in with Pydantic")

Schema defined!
   Fields: ['summary', 'key_points', 'pros', 'cons', 'confidence']
   Validation: Built-in with Pydantic


### Common Mistakes with Pydantic Schemas

When defining schemas, watch out for these common pitfalls:

- **Mistake 1: Missing Field Descriptions**
```python
class BadSchema(BaseModel):
    items: List[str]  # No description - LLM doesn't know what to put here!
```

- **Good Practice:**
```python
class GoodSchema(BaseModel):
    items: List[str] = Field(description="List of main points from the analysis")
```

- **Mistake 2: Vague Field Names**
```python
class BadSchema(BaseModel):
    data: str  # What data? Too vague!
```

- **Good Practice:**
```python
class GoodSchema(BaseModel):
    executive_summary: str = Field(description="1-2 sentence overview")
```

- **Mistake 3: No Validation Constraints**
```python
class BadSchema(BaseModel):
    score: float  # Could be negative or > 100!
```

- **Good Practice:**
```python
class GoodSchema(BaseModel):
    score: float = Field(description="Quality score", ge=0, le=100)
```

**Pro Tip:** The better your schema description, the better the LLM's output!

## Step 3: Create LLM with Structured Output

We use `.with_structured_output()` to bind the schema to the LLM:

In [62]:
# Create base LLM
# Use get_chat_model() - uses Holistic AI Bedrock by default (recommended)
llm = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)

# Create LLM with structured output
llm_structured = llm.with_structured_output(TechAnalysis)
print("✅ LLM with structured output created!")
print("   Model: claude-3-5-sonnet (via Bedrock)")
print("   Output format: TechAnalysis (JSON)")
print("   Validation: Automatic with Pydantic")


✅ LLM with structured output created!
   Model: claude-3-5-sonnet (via Bedrock)
   Output format: TechAnalysis (JSON)
   Validation: Automatic with Pydantic


## Step 4: Test Structured Output

Let's ask the LLM to analyze a technology and get validated JSON:

In [63]:
# Test structured output
question = "Analyze quantum computing technology"

print(f"Question: {question}\n")
print("="*70)

start_time = time.time()
result = llm_structured.invoke(question)
elapsed = time.time() - start_time

print("Structured Output (Pydantic Object):")
print("="*70)
print(f"Summary: {result.summary}\n")
print(f"Key Points ({len(result.key_points)}):")
for i, point in enumerate(result.key_points, 1):
    print(f"  {i}. {point}")

print(f"\nPros ({len(result.pros)}):")
for i, pro in enumerate(result.pros, 1):
    print(f"  {i}. {pro}")

print(f"\nCons ({len(result.cons)}):")
for i, con in enumerate(result.cons, 1):
    print(f"  {i}. {con}")

print(f"\nConfidence: {result.confidence}%")
print("="*70)
print(f"Time: {elapsed:.2f}s")
print(f"\nType: {type(result).__name__} (automatically validated!)")

Question: Analyze quantum computing technology



Structured Output (Pydantic Object):
Summary: Quantum computing leverages quantum mechanical phenomena to perform complex calculations exponentially faster than classical computers for specific applications.

Key Points (5):
  1. Utilizes quantum bits (qubits) that can exist in multiple states simultaneously through superposition
  2. Could revolutionize fields like cryptography, drug discovery, and optimization problems
  3. Currently limited by challenges in maintaining qubit coherence and error correction
  4. Major tech companies and governments are investing heavily in quantum computing research
  5. May achieve quantum advantage for specific tasks within this decade

Pros (3):
  1. Exponentially faster processing for certain algorithms and problems
  2. Potential to break current encryption methods and develop new secure communications
  3. Could enable breakthroughs in materials science and drug development

Cons (3):
  1. Extremely sensitive to environmental interference and re

## Step 5: Compare Natural Text vs Structured Output

Let's see the difference between natural text and structured output:

In [64]:
print("="*70)
print("COMPARISON: Natural Text vs Structured Output")
print("="*70)

# Natural text response
result_natural = llm.invoke(question)

print("\nNatural Text Output:")
print("-"*70)
print(result_natural.content[:300] + "...")
print("-"*70)
print("Human-readable")
print("Conversational")
print("ERROR: Hard to parse programmatically")
print(f"Type: {type(result_natural.content).__name__}")

print("\nStructured Output:")
print("-"*70)
result_json = result.model_dump_json(indent=2)
print(result_json[:300] + "...")
print("-"*70)
print("Easy to parse and validate")
print("Type-safe with Pydantic")
print("Perfect for APIs and databases")
print("ERROR: Less conversational")
print(f"Type: {type(result).__name__}")

COMPARISON: Natural Text vs Structured Output

Natural Text Output:
----------------------------------------------------------------------
Here's a detailed analysis of quantum computing technology:

Core Concepts:

1. Quantum Bits (Qubits)
- Unlike classical bits (0 or 1), qubits can exist in multiple states simultaneously (superposition)
- Can be entangled with other qubits
- Enables parallel processing of vast amounts of data

2. Ke...
----------------------------------------------------------------------
Human-readable
Conversational
ERROR: Hard to parse programmatically
Type: str

Structured Output:
----------------------------------------------------------------------
{
  "summary": "Quantum computing leverages quantum mechanical phenomena to perform complex calculations exponentially faster than classical computers for specific applications.",
  "key_points": [
    "Utilizes quantum bits (qubits) that can exist in multiple states simultaneously through superposi...
-------------

## Step 6: Working with Structured Data

Once you have a Pydantic object, you can easily access fields and convert formats:

In [57]:
print("Accessing Pydantic object fields:\n")

# Direct field access (with autocomplete!)
print(f"Summary: {result.summary[:100]}...")

# Convert to dictionary
as_dict = result.model_dump()
print(f"\nAs dictionary: {type(as_dict).__name__} with {len(as_dict)} keys")

# Convert to JSON string
as_json = result.model_dump_json(indent=2)
print(f"\nAs JSON string ({len(as_json)} characters):")
print(as_json[:200] + "...")

# Save to file (example)
with open('/tmp/tech_analysis.json', 'w') as f:
    f.write(as_json)
print("\nSaved to /tmp/tech_analysis.json")

# Load and validate from JSON
with open('/tmp/tech_analysis.json', 'r') as f:
    loaded_data = json.load(f)
    validated = TechAnalysis(**loaded_data)
    print(f"\nLoaded and validated: Confidence = {validated.confidence}%")

Accessing Pydantic object fields:

Summary: Quantum computing leverages quantum mechanical phenomena to perform complex calculations exponential...

As dictionary: dict with 5 keys

As JSON string (1080 characters):
{
  "summary": "Quantum computing leverages quantum mechanical phenomena to perform complex calculations exponentially faster than classical computers for specific applications.",
  "key_points": [
  ...

Saved to /tmp/tech_analysis.json

Loaded and validated: Confidence = 85.0%


### Try It Yourself!

Create your own schema for a different use case. Here are some ideas:

**Idea 1: Movie Review Analysis**
```python
class MovieReview(BaseModel):
    title: str = Field(description="Movie title")
    rating: float = Field(description="Rating out of 5 stars", ge=0, le=5)
    genres: List[str] = Field(description="List of genres")
    plot_summary: str = Field(description="Brief plot summary")
    recommendation: str = Field(description="Who should watch this movie")
```

**Idea 2: Product Comparison**
```python
class ProductComparison(BaseModel):
    product_name: str = Field(description="Product being analyzed")
    price_range: str = Field(description="Price category: budget/mid/premium")
    features: List[str] = Field(description="Key features")
    best_for: str = Field(description="Target audience or use case")
    alternatives: List[str] = Field(description="Similar products")
```

**Exercise:** Try creating a schema for analyzing restaurant reviews, then test it!

## Step 7: Using Structured Output with Agents

### The Proper Way: `response_format` Parameter

Good news! You **CAN** use structured output with `create_react_agent` by using the `response_format` parameter:

```python
agent = create_react_agent(
    llm,                           # Regular LLM (not .with_structured_output())
    tools=[my_tool],               # Your tools
    response_format=MySchema       # Your Pydantic schema
)
```

**Key Differences:**

| Approach | Code | Result Location |
|----------|------|-----------------|
| **Direct LLM** | `llm.with_structured_output(Schema)` | Returns Pydantic object |
| **Agent** | `create_react_agent(..., response_format=Schema)` | `result['structured_response']` |

### How It Works

1. Agent uses tools to gather information
2. Agent generates a natural language response
3. A **second LLM call** structures the response according to your schema
4. You get both: `messages` (conversation) and `structured_response` (Pydantic object)

Let's see it in action:

In [58]:
from langchain_core.tools import tool

# Define a tool
@tool
def get_tech_info(tech_name: str) -> str:
    """Get detailed information about a technology.
    
    Args:
        tech_name: Name of the technology
        
    Returns:
        Detailed technical information
    """
    info_db = {
        "quantum computing": "Uses quantum mechanics principles like superposition and entanglement. Current systems are NISQ (noisy intermediate-scale quantum). Major players: IBM, Google, Rigetti. Applications: cryptography, drug discovery, optimization problems.",
        "blockchain": "Distributed ledger with consensus mechanisms. Smart contracts enable programmable transactions. Proof-of-work vs proof-of-stake. Applications: cryptocurrency, supply chain tracking, decentralized identity.",
        "ai": "Machine learning, deep neural networks, transformers. GPT models for NLP, CNNs for vision, reinforcement learning for control. Applications: language processing, image recognition, autonomous systems."
    }
    return info_db.get(tech_name.lower(), "Technology not in database")

# Create agent with response_format
agent_with_tools = create_react_agent(
    llm,                          # Pass regular LLM
    tools=[get_tech_info],        # Tools for the agent
    response_format=TechAnalysis  # Schema for structured output
)

print("Agent created with tools AND structured output!")
print("="*70)

# Invoke the agent
result = agent_with_tools.invoke({
    "messages": [("user", "Analyze quantum computing technology. Use the get_tech_info tool to gather information.")]
})

# Access the structured response
structured = result['structured_response']

print("\nStructured Response (TechAnalysis object):")
print("="*70)
print(f"Type: {type(structured).__name__}")
print(f"\nSummary: {structured.summary}")
print(f"\nKey Points ({len(structured.key_points)}):")
for i, point in enumerate(structured.key_points, 1):
    print(f"  {i}. {point}")

print(f"\nPros ({len(structured.pros)}):")
for i, pro in enumerate(structured.pros, 1):
    print(f"  {i}. {pro}")

print(f"\nCons ({len(structured.cons)}):")
for i, con in enumerate(structured.cons, 1):
    print(f"  {i}. {con}")

print(f"\nConfidence: {structured.confidence}%")

# Also show tool usage
print("\nTool Calls Made:")
print("="*70)
tool_calls = [msg for msg in result['messages'] if hasattr(msg, 'tool_calls') and msg.tool_calls]
if tool_calls:
    for msg in tool_calls:
        for tc in msg.tool_calls:
            print(f"  - {tc['name']}({tc['args']})")
else:
    print("  No tools were called")

print("\nTIP: Pattern: Agent uses tools → Gathers info → Returns structured output")


Agent created with tools AND structured output!

Structured Response (TechAnalysis object):
Type: TechAnalysis

Summary: Quantum computing is an emerging technology that leverages quantum mechanical principles like superposition and entanglement to solve complex computational problems beyond classical computing capabilities.

Key Points (4):
  1. Currently in NISQ (Noisy Intermediate-Scale Quantum) era with significant error correction challenges
  2. Major tech companies like IBM, Google, and Rigetti are actively developing quantum systems
  3. Primary applications include cryptography, drug discovery, and complex optimization problems
  4. Operates using quantum mechanical principles fundamentally different from classical computing

Pros (3):
  1. Potential to solve previously intractable computational problems
  2. Wide range of applications in security, science, and optimization
  3. Strong industry investment and development from major tech companies

Cons (3):
  1. Still in early

## Troubleshooting Common Issues### Issue 1: Validation Error```pydantic.ValidationError: 1 validation error for TechAnalysisconfidence  Input should be less than or equal to 100```**Solution:** The LLM returned a value outside your constraints. Check your Field() constraints (ge, le, etc.)### Issue 2: Missing Required Fields```pydantic.ValidationError: Field required```**Solution:** Make fields optional with `Optional[...]` or provide defaults:```pythonfrom typing import Optionalclass FlexibleSchema(BaseModel):    required_field: str    optional_field: Optional[str] = None    field_with_default: int = 0```### Issue 3: LLM Returns Text Instead of JSON```JSONDecodeError: Expecting value```**Solution:** Ensure you're using `.with_structured_output()` and that the model supports function calling (claude-3-5-sonnet does!).### Issue 4: Output is Too SlowStructured output requires the LLM to format JSON, which can be slower.**Solution:** - Use structured output only when you need it- For simple extractions, consider using natural text with regex parsing- For production, consider caching results### Debugging Tips1. **Check the schema:** Print `YourSchema.model_json_schema()` to see what the LLM receives2. **Test with simple prompts:** Start with easy questions before complex ones3. **Validate manually:** Try `YourSchema(**test_data)` with sample data4. **Check field descriptions:** Make sure they're clear and specific

## Hands-On Exercise

Try this exercise to practice what you've learned:

### Task: Build a News Article Analyzer

Create a Pydantic schema and LLM that analyzes news articles and returns:
- Article category (politics/tech/sports/entertainment/other)
- Sentiment (positive/negative/neutral)
- Key entities mentioned (people, organizations, locations)
- Main topics (3-5 bullet points)
- Reading difficulty (easy/medium/hard)

**Starter Code:**
```python
class NewsAnalysis(BaseModel):
    category: str = Field(description="Article category")
    sentiment: str = Field(description="Overall sentiment")
    # Add more fields here!

llm_news = llm.with_structured_output(NewsAnalysis)
result = llm_news.invoke("Analyze this article: [paste article text]")
```

**Bonus Challenge:** Add validation to ensure category is one of the valid options using Pydantic's `Literal` type!

## SummaryCongratulations! You've learned:### Key Concepts1. **Pydantic Schemas** - Define structured output formats with validation2. **with_structured_output()** - Bind schemas to LLMs for automatic JSON parsing3. **Type Safety** - Get validated Pydantic objects instead of raw strings4. **Comparison** - Understand trade-offs between natural text and structured output### When to Use Each#### Natural Text (Default)- Chatbots and conversational interfaces- Reports and explanations- Human-facing applications- When flexibility is needed#### Structured Output (with Pydantic)- API responses- Data extraction and transformation- Database storage- Integration with other systems- When type safety and validation matter### Important Notes- Structured output may be slightly slower due to JSON formatting- The LLM must support function calling (claude-3-5-sonnet does!)- Validation happens automatically with Pydantic- Field descriptions help the LLM understand what to generate---**Pro Tip**: Define reusable Pydantic schemas for common tasks (e.g., `SearchResult`, `Analysis`, `Summary`) and reuse them across your application!